# Multi-tenant Chat with Papers - Setup
## Get keys and urls

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])

cwzujlbgtk
L1FUbzRPaU


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

True

## Create Tenant-ready collection

In [3]:
from weaviate.classes.config import Configure

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",

    vector_config=[
        Configure.Vectors.text2vec_weaviate(
            name="main_vector",

            model="Snowflake/snowflake-arctic-embed-l-v2.0",
            source_properties=["chunk"]
        )
    ],

    # supported models: https://weaviate.io/developers/weaviate/model-providers/openai/generative#available-models
    generative_config=Configure.Generative.openai(
        model="gpt-4o-mini" # gpt-4
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [4]:
papers = client.collections.get("Papers")
papers.tenants.get()

{}

## Close the client

In [5]:
client.close()